In [1]:
import math
from tensorboardX import SummaryWriter

sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import env_configurations
import games_configurations
env_name = 'SuperMarioBrosRandomStages-v1'#'PongNoFrameskip-v4' #'MountainCar-v0' #'SuperMarioBros-v1'# 'PongNoFrameskip-v4' #SuperMarioBros-v1'
#env_name = 'CartPole-v1'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #

obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
config = games_configurations.mario_random_config_lstm

observation_shape = obs_space.shape
n_actions = action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

(None, 84, 84, 4)
7


In [4]:
from a2c_discrete import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

2019-07-20 18:24:28,947	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_18-24-28_17599/logs.
2019-07-20 18:24:29,059	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:21191 to respond...
2019-07-20 18:24:29,186	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:61018 to respond...
2019-07-20 18:24:29,192	INFO services.py:804 -- Starting Redis shard with 6.74 GB max memory.
2019-07-20 18:24:29,264	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_18-24-28_17599/logs.
2019-07-20 18:24:29,267	INFO services.py:1427 -- Starting the Plasma object store with 10.11 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:21191',
 'object_store_address': '/tmp/ray/session_2019-07-20_18-24-28_17599/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-20_18-24-28_17599/sockets/raylet',
 'webui_url': None}

In [ ]:


agent = A2CAgent(sess,'stages_all', obs_space, True, action_space, config)
#agent.restore('nn/last_skip')
agent.train()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


2019-07-20 18:24:30,856	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


0
1
2
3
4
5
6
7
0
(pid=17703) 
(pid=17703) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=17703) For more information, please see:
(pid=17703)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=17703)   * https://github.com/tensorflow/addons
(pid=17703) If you depend on functionality not listed there, please file an issue.
(pid=17703) 
(pid=17698) 
(pid=17698) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=17698) For more information, please see:
(pid=17698)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=17698)   * https://github.com/tensorflow/addons
(pid=17698) If you depend on functionality not listed there, please file an issue.
(pid=17698) 
(pid=17701) 
(pid=17701) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=17701) For more information, please see:
(pid=17701)   * https://github.com

In [ ]:
agent.save('nn/last_all')
ray.shutdown()

In [ ]:
gym.envs.registry.all()